# Apriori Algorithm Recommendation System

In [1]:
import numpy as np

## Preprocessing


In [2]:
# read the data from the input file /data/video_games.txt 
data = np.zeros((4096, 47), dtype=np.dtype('U60'))

with open('./data/video_games.txt', 'r') as video_games_file:
    for line_idx, line in enumerate(video_games_file):
        line = line.rstrip()
        games = line.split(';')
        for game_idx, game in enumerate(games):
            data[line_idx][game_idx] = game

In [3]:
# determine the unique items and create a dictionary that maps an item to its id
unique_items = np.unique(data)
unique_items = np.delete(unique_items, 0, axis=0) 

id_to_item = unique_items

item_to_id = {}
for idx, item in enumerate(id_to_item):
    item_to_id[item] = idx

item_to_id

{"Assassin's Creed Origins": 0,
 'Bastion': 1,
 'Bloodborne': 2,
 'Chrono Trigger': 3,
 'Code Vein': 4,
 'Crosscode': 5,
 'Cyberpunk 2077': 6,
 'Dark Souls 3': 7,
 'Darkest Dungeon': 8,
 'Diablo III': 9,
 'Dragon Age: Inquisition': 10,
 'Dragon Age: Origins': 11,
 'Dragons Dogma: Dark Arisen': 12,
 'Dying Light 2 Stay Human': 13,
 'Elden Ring': 14,
 'Fable II': 15,
 'Fallout 4': 16,
 'Fallout New Vegas': 17,
 'Fire Emblem: Three Houses': 18,
 'Genshin Impact': 19,
 'Greedfall': 20,
 'Grim Dawn': 21,
 'Hades': 22,
 'Hogwarts Legacy': 23,
 'Horizon Zero Dawn': 24,
 'Kingdom Come: Deliverance': 25,
 'Kingdom Hearts 3': 26,
 'Legend of Grimrock 2': 27,
 'Mass Effect Legendary Edition': 28,
 'Middle-Earth: Shadow of War': 29,
 'NieR: Automata': 30,
 'Path Of Exile': 31,
 'Persona 5 Strikers': 32,
 'Pillars of Eternity': 33,
 'Scarlet Nexus': 34,
 'Soulstice': 35,
 'Stardew Valley': 36,
 'Tales of Arise': 37,
 'Tales of Berseria': 38,
 'The Last Oricru': 39,
 'The Legend of Zelda: Breath of 

In [4]:
data = []

with open('./data/video_games.txt', 'r') as video_games_file:
    for line in video_games_file:
        line = line.rstrip()
        games = line.split(';')
        line_data = []
        for game in games:
            line_data.append(game)
        data.append(line_data)
    

In [5]:
# map the items of the records to ids and sort each record
mapped_records = [[item_to_id.get(item) for item in sorted(row)] for row in data]
mapped_records

[...]

## Apriori algorithm implementation

### Determine the frequent 1 itemsets


In [6]:
# calculate the support of length-1 itemsets using Counter or defaultdict
from collections import Counter
l1_items = {}

flattened_records = [item for line in mapped_records for item in line]

counter = Counter(flattened_records)

for item, count in counter.items():
    l1_items[item] = count

In [7]:
# filter out the frequent length-1 itemsets with their support
frequent_l1_items = {key: count for key, count in l1_items.items() if count >= 226 and key != None}
frequent_l1_items

{0: 1035,
 1: 373,
 4: 1556,
 20: 303,
 22: 348,
 27: 246,
 31: 378,
 32: 339,
 33: 269,
 41: 620,
 44: 758,
 11: 238,
 13: 1489,
 39: 251,
 40: 298,
 45: 248,
 26: 415,
 43: 980,
 3: 228,
 12: 680,
 14: 1648,
 18: 264,
 28: 318,
 34: 774,
 35: 232,
 38: 684,
 46: 250,
 2: 379,
 15: 248,
 19: 1046,
 29: 667,
 10: 329,
 24: 340,
 5: 363,
 25: 753,
 17: 389,
 42: 303,
 37: 390,
 6: 1539,
 21: 431,
 23: 1654,
 47: 319,
 16: 1383,
 48: 633,
 9: 1052,
 7: 1104,
 36: 230}

In [8]:
# save the length-1 frequent items to frequent_items with their support
frequent_items = frequent_l1_items

### Determine the frequent n itemsets

In [9]:
# implement the apriori_gen algorithm based on the lecture slides
def apriori_gen(itemsets):
    k = len(itemsets[0])
    candidates = []
    # generate candidates
    for p in range(len(itemsets)):
        for q in range(p+1, len(itemsets)):
            if k > 1:
                subset_p = itemsets[p][:-1]
                subset_q = itemsets[q][:-1]
            else:                                 
                subset_p = itemsets[p]
                subset_q = itemsets[q]

            if k>1 and subset_p == subset_q:
                candidate = subset_p + (itemsets[p][-1],) + (itemsets[q][-1],)
                candidates.append(candidate)
            
            elif k==1:
                candidate = subset_p + subset_q
                candidates.append(candidate)
            
    # prune the candidates and return them 
        # split into substrings and check all if frequent
        if k > 1:
            for candidate in candidates:
                for i in range(len(candidate)):
                    subset = candidate[:i] + candidate[i+1:]
                    if (subset not in itemsets) and (candidate in candidates): candidates.remove(candidate)
    
    return candidates

In [10]:
# implement an algorithm to calculate the support of the given itemset
def calculate_support(itemset):
    support = 0
    for row in mapped_records:
        contains_all_items = True
        for item in itemset:
            if item not in row: contains_all_items = False
        if contains_all_items == True: support = support + 1

    return support

In [11]:
# set the initial candidates which will be used to generate the frequent length-2 itemsets
candidates = [(key,) for key in frequent_l1_items.keys()]

while True:
    new_candidates = apriori_gen(candidates)
    supports = map(calculate_support, new_candidates)

    # filter out the frequent candidates
    frequent_new_candidates = {}
    for idx, supp in enumerate(supports):
        if supp > 226 : frequent_new_candidates[new_candidates[idx]] = supp

    if len(frequent_new_candidates) == 0 : break

    # add the frequent candidates to frequent_items 
    frequent_items.update(frequent_new_candidates)

    # replace candidates with the new ones
    candidates = [itemset for itemset in frequent_new_candidates.keys()]

### Save results


In [12]:
with open('patterns.txt', 'w') as patterns:
    for ids in frequent_items.keys():
        support = frequent_items.get(ids)
        
        games = ""
        if isinstance(ids, int):
            games = id_to_item[ids] + ";"
        else:
            for id in ids:
                games = games + id_to_item[id] + ";"

        patterns.write(str(support) + ':' + games[:-1] + '\n')

## Recommendation Example

In [13]:
game1 = item_to_id.get("Elden Ring")
game2 = item_to_id.get("Scarlet Nexus")

relevant_items = {}
for itemset in frequent_items.keys():
    if isinstance(itemset, int) or len(itemset)<3 : continue
    if game1 in itemset and game2 in itemset:
        relevant_items[itemset] = frequent_items.get(itemset)

relevant_items

{(4, 14, 34): 232, (13, 14, 34): 233, (14, 34, 6): 249, (14, 34, 23): 249}

In [14]:
id_to_item[6]

'Cyberpunk 2077'

In [15]:
id_to_item[23]

'Hogwarts Legacy'

In [16]:
conf = 249 / calculate_support([4,34])
f"Confidence: {conf}"

'Confidence: 0.7073863636363636'

$conf(\textrm{Elden Ring} \cup \textrm{Scarlet Nexus} \Rightarrow \textrm{Hogwarts Legacy}) = \frac{supp(\textrm{Elden Ring} \cup \textrm{Scarlet Nexus} \cup \textrm{Hogwarts Legacy})}{supp(\textrm{Elden Ring} \cup \textrm{Scarlet Nexus})}$

(analogically for Cyberpunk 2077)
